In [1]:
from modelUXception import *
from data import *

vid="v020" #version id should match the file number 
# Ran on DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'mass_seg_05/train05','mass_seg','mass_seg_labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_05/valid05','mass_seg','mass_seg_labels',data_gen_args,save_to_dir = None)

# 300 images are used for training, 52 images for validating, and 60 images for testing
train_steps = 300//batch_size
valid_steps = 52//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100

model = UXception(start_neurons=16)

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files05/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files05/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 300 images belonging to 1 classes.
Found 300 images belonging to 1 classes.
Epoch 1/100
450/450 [==============================] - ETA: 0s - loss: 0.8542 - acc: 0.9024 - dice_coef: 0.1458Found 52 images belonging to 1 classes.
Found 52 images belonging to 1 classes.
450/450 [==============================] - 284s 555ms/step - loss: 0.8542 - acc: 0.9024 - dice_coef: 0.1458 - val_loss: 0.7311 - val_acc: 0.9684 - val_dice_coef: 0.2689

Epoch 00001: saving model to files05\unet_mass_seg_v020.hdf5


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
450/450 [==============================] - 245s 545ms/step - loss: 0.7159 - acc: 0.9761 - dice_coef: 0.2841 - val_loss: 0.6474 - val_acc: 0.9852 - val_dice_coef: 0.3526

Epoch 00002: saving model to files05\unet_mass_seg_v020.hdf5
Epoch 3/100
450/450 [==============================] - 244s 543ms/step - loss: 0.6333 - acc: 0.9806 - dice_coef: 0.3667 - val_loss: 0.6284 - val_acc: 0.9783 - val_dice_coef: 0.3716

Epoch 00003: saving model to files05\unet_mass_seg_v020.hdf5
Epoch 4/100
450/450 [==============================] - 243s 541ms/step - loss: 0.5640 - acc: 0.9839 - dice_coef: 0.4360 - val_loss: 0.6284 - val_acc: 0.9789 - val_dice_coef: 0.3716

Epoch 00004: saving model to files05\unet_mass_seg_v020.hdf5
Epoch 5/100
450/450 [==============================] - 244s 542ms/step - loss: 0.5287 - acc: 0.9846 - dice_coef: 0.4713 - val_loss: 0.5457 - val_acc: 0.9809 - val_dice_coef: 0.4543

Epoch 00005: saving model to files05\unet_mass_seg_v020.hdf5
Epoch 6/100
450/450 [=======

### validate your model and save predicted results

In [3]:
validGene = testGenerator("mass_seg_05/valid05/pred")
model.load_weights("files05/unet_mass_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,52,verbose=1)
saveResult("mass_seg_05/valid05/pred",results,vid)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


52/52 [==============================] - 24s 394ms/step


D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\0_predict_v020.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\1_predict_v020.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\2_predict_v020.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\3_predict_v020.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\7_predict_v020.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"